# Interactive histogramming

In this example, we have a dataset which contains information about the number of taxi ride drop-offs in New York City in the year 2015.
The raw data (obtained from the [Vaex documentation](https://vaex.readthedocs.io/en/latest/datasets.html))
has been histogrammed into three dimensions: latitude, longitude and hour-of-the-day.

The goal is to create an interactive visualization with on-the-fly histogramming and smoothing.
It will contain:

- a central 2D image that shows a longitude-latitude map of the drop-offs, for a single hour-of-the-day slice.
- a slider above the main panel which allows to navigate the hour-of-the-day dimension, updating the 2D map.
- a 1D figure below the 2D map, displaying the same data but summed along the latitude dimension.
- in the 1D panel, we also want to add a gaussian smoothed version of the 1D curve.

In [ ]:
%matplotlib widget
import scipp as sc
import plopp as pp
from plopp import widgets
from plopp.data import examples
try:
    from scipp.scipy.ndimage import gaussian_filter
except ImportError:
    from scipp.ndimage import gaussian_filter
import ipywidgets as ipw

data = sc.io.open_hdf5(examples.nyc_taxi())
data

In [ ]:
# Input data node
in_node = pp.input_node(data)
in_node.name = 'Input data'

# Slider node that provides index to slice
slider = ipw.IntSlider(description='Hour:', min=0, max=23)
slider_node = pp.widget_node(slider)
slider_node.name = 'Slider widget'

# Node that actually does the slicing
slice_node = pp.node(
    lambda da, ind: da['hour', ind])(da=in_node, ind=slider_node)
slice_node.name = 'Slice by hour'

# Add a 2D figure to show the NYC map
fig2d = pp.figure2d(slice_node, norm='log')

# Add a node after the slicing to sum along the latitude dimension
sum_lat = pp.node(sc.sum, dim='latitude')(slice_node)
sum_lat.name = 'Sum along latitude'

# Add a node after the sum that performs as Gaussian smoothing
smooth = pp.node(gaussian_filter, sigma=5)(sum_lat)
smooth.name = 'Gaussian smoothing'

# Add a 1D figure that will display both raw sum and smoothed data
fig1d = pp.figure1d(sum_lat, smooth, norm='log')

widgets.Box([slider, fig2d, fig1d])  # Container box

Below is a preview of the graph of connected nodes:

In [ ]:
pp.show_graph(in_node)

In [ ]:
# This cell is used to generate the thumbnail for the docs gallery.
# It is hidden from the online documentation.
fig2d.ax.set_aspect('equal')
fig2d.save('../../_static/gallery/nyc-taxi-thumbnail.png')